In [ ]:
MYSQL INTRODUCTION 
=====================================

-> Mysql is a database management system.
-> Database is a structured collection of data.
-> To add , access and process data in a computer database , we need a database management system such as Oracle 
   , Mysql.

-> Mysql database is relational . 
-> A relational database stores data in seperate tables rather than putting all in one big file.
-> Rules are defined between different tables with related fields.

-> Mysql is open source . Mysql can be downloaded from internet and can be used without paying.
-> Mysql server is fast , reliable , scalable and easy to use.

In [ ]:
MYSQL VERSIONS 
===========================================

-> bugfix series like Mysql 8.4 
-> innovation release like Mysql 8.3

-> both relase contains bug fixes , however innovation release includes new feature.
-> both are meant for production use.


MYSQL 8.0.34 
-> 8 is the major version.
-> 0 is the minor version.
-> 34 is the bugfix release version.

In [ ]:
INSTALLING MYSQL ON LINIX USING GENERIC BINARIES
=============================================================


-> Download the TAR binary from the MySQL Downloads page, at https://dev.mysql.com/downloads/mysql/, 
   then choose Linux - Generic as the OS and select the version. You can download directly onto your 
   server  directly using the wget command:

  wget "https://downloads.mysql.com/archives/get/p/23/file/mysql-8.0.20-linux-glibc2.12-x86_64.tar.xz"


-> MySQL has a dependency on the libaio library. The data directory initialization, and subsequent server 
   startup steps, will fail if this library is not installed locally.

   yum install -y libaio


-> The OS 8 platforms by default do not install the file /lib64/libtinfo.so.5, which is required by the
   MySQL client  bin/mysql , To work around this issue, install the ncurses-compat-libs package.

   yum install ncurses-compat-libs


-> Add the mysql group and the mysql user. All the files and directories should be under the mysql user

   groupadd mysql
   useradd -r -g mysql -m-s /bin/false mysql
   cd  /home/mysql
   tar -zxvf mysql-8.0.20-linux-glibc2.12-x86_64.tar.xz
   cd /usr/local
   ln -s /home/mysql/mysql-8.0.20-linux-glibc2.12-x86_64 mysql  # make symbolic link
   export PATH=$PATH:/usr/local/mysql/bin


-> Create seperate directories for data , redolog , binlog , errorlog and systemdata 

   mkdir -p /opt/mysql/data
   mkdir -p /opt/mysql/redolog
   mkdir -p /opt/mysql/undolog
   mkdir -p /opt/mysql/systemdata
   mkdir -p /opt/mysql/mysql-files
 
   chown -R mysql:mysql /opt/mysql/
   chmod 750 /opt/mysql/mysql-files


-> Start mysqld by specfying datadir and user mysql 

   mysqld --initialize --datadir=/opt/mysql/data --user=mysql

   mysql_ssl_rsa_setup --datadir=/opt/mysql/data/

   mysqld_safe --user=mysql --datadir=/opt/mysql/data &

   mysql_secure_installation 


-> Copy startup files to init.d for auto start and update the datadir in the file

   cp support-files/mysql.server /etc/init.d/mysql.server

   vi /etc/init.d/mysql.server  -- datadir=/opt/mysql/data

   /etc/init.d/mysql.server stop
   /etc/init.d/mysql.server start


-> Create option/configuration file by adding different options

   vi /etc/mysql/my.cnf

    [mysqld]

    innodb-data-file-path = ibdata1:12M:autoextend
    innodb-data-home-dir = /opt/mysql/systemdata/

    innodb-undo-directory = /opt/mysql/undolog

    innodb-log-group-home-dir = /opt/mysql/redolog

    innodb-temp-data-file-path = /opt/mysql/tempdata/ibtmp1:12M:autoextend

    log-bin = /opt/mysql/binlog/prod01_binlog
    log-bin-index = /opt/mysql/binlog/prod01_binlog.index

    log-error = /opt/mysql/errorlog/prod01_mysqld.log

    server-id = 1
    

-> move files to respective directories and start the mysqld 

    [root@mysqlprod01 data]# mv ibdata1 ../systemdata/
    [root@mysqlprod01 data]# mv ib_logfile* ../redolog/

    mysqld_safe --user=mysql --datadir=/opt/mysql/data --syslog &

    mysql> select @@hostname , @@version;

In [ ]:
Installing MySQL on Linux Using the MySQL Yum Repository
 =======================================================================

 https://dev.mysql.com/downloads/repo/yum/

 
 -> install the repo locally and check /etc/yum.repos.d/ directory

     yum localinstall mysql84-community-release-el8-1.noarch.rpm

 -> check which version is enabled

    yum repolist enabled | grep mysql.*-community

 -> if you want to install 8.0 , then disable 8.4 and enable 8.0

    yum-config-manager --disable mysql-8.4-lts-community

    dnf config-manager --enable  mysql80-community  ( dnf is a newer tool similar to yum )

 -> install 8.0 community server

    yum install mysql-community-server


 -> start MySql service and enable auto-start 

      systemctl start mysqld
      systemctl enable mysqld.service

 -> change password of root user 

    grep 'temporary password' /var/log/mysqld.log
    mysql -u root -p


 -> verify whether mysql is running

    systemctl status mysqld
    pidof mysqld
    netstat -ntlp | grep 3306
    lsof -u mysql   ( list open files by user mysql)

In [ ]:
Installing MySQL on Linux Using  RPM Packages
=======================================================================

-> download locally the RPM packages from mysql site.

     mysql-community-server-8.0.20-1.el8.x86_64.rpm
     mysql-community-client-8.0.20-1.el8.x86_64.rpm
     mysql-community-libs-8.0.20-1.el8.x86_64.rpm
     mysql-community-devel-8.0.20-1.el8.x86_64.rpm
     mysql-community-common-8.0.20-1.el8.x86_64.rpm
     mysql-community-icu-data-files-8.0.20-1.el8.x86_64.rpm

-> install the packages individually or all at once.

    yum localinstall mysql-community-*.rpm

-> check the installed packages and start the server

    rpm -qa | grep mysql
    systemctl start mysqld
    systemctl enable mysqld

In [ ]:
Installing MySQL on Linux With Dokcer Containers
=======================================================================

-> add users to docker group 

   usermod -a -G docker mysqluser
   id mysqluser

-> search and pull mysql image 

   docker images
   docker ps
   docker search mysql
   docker pull mysql   ( for latest )
   docker pull mysql/mysql-server:8.0.30    ( for specific version )
   docker images

-> run mysql container 

   docker run --name mysql_8030 -e MYSQL_ROOT_PASSWORD=Root@2024 -d mysql/mysql-server:8.0.30

   dokcer ps
   docker logs <container_name>


-> go into docker mysql host

   docker exec -it <container_name> bash 
   mysql -u root -p
   select @@hostname , @@version


->  run mysql container exposing the port 3306 from container to host. 
    by this outside client can be connected to mysql which is running in the container.

    docker run --name mysql_8030 -e MYSQL_ROOT_PASSWORD=Root@2024 -d -p 3306:3306 mysql/mysql-server:8.0.30
               
-> run mysql container by binding local volume to container volume.
   by this even if the docker container is removed the local volume will not be destroyed.
   you can start new mysql container with this data.

   docker run --name mysql_8030 -e MYSQL_ROOT_PASSWORD=Root@2024 -d -p 3306:3306
   -v /var/lib/mysql:/var/lib/mysql mysql/mysql-server:8.0.30

In [ ]:
MySQL Upgrades 
===============================

Minor version upgarde -  8.0.28 -> 8.0.30  , in-place

Major version upgrade -  5.7 -> 8.0 , remove old and install new

-> Upgrade from 5.7 to 8.0 is suppoerted.
-> Upgrade is supported only between General Availabilty (GA).
-> Skipping a release is also supported ( 8.0.21 -> 8.0.23).


Upgrade Types 
===================

-> in-place upgrade - replacing the mysql server packages.
-> logical upgrade - exporting sql from old mysql instance into new.
-> replication topology upgrade.


Parts upgraded during the upgrade 
===========================================

-> Data dictionary upgrade - data dictionary tables in mysql , performance_schema , information_schema
                             and ndbinfo.

-> Server upgrade - system tables in mysql , sys schema and user schemas.

-> prior to 8.0.16 an extra step of mysql_upgrade has to be run by DBA.

-> After 8.0.16 the server itself performs all the tasks handled by mysql_upgrade.

In [ ]:
Minor Version Upgrade From 8.0.30 -> 8.0.35 ( rpm pacakges )
===============================================================

-> check issues in the current database objects if any 

     mysqlcheck --all-databases --check-upgrade

-> check the target version is valid for upgrade

   mysqlsh -- util check-for-server-upgrade root@localhost --target-version=8.0.35 --output-format=JSON 
              --config-path=/etc/my.cnf

-> Download 8.0.35 rpm packages 

->  Stop the mysqld server and install the packages

     systemctl stop mysqld

     yum localinstall mysql-community-*

-> verify the installed pacakges 

   rpm -qa | grep mysql

->  Start the mysqld server and check 
  
     systemctl start mysqld
     systemctl status mysqld
     pidof mysqld
     netstat -ntlp | grep 3306


In [ ]:
Major version upgrade from 5.7 to 8.30 ( binary )
=================================================================

-> check issues in the current database objects if any 

     mysqlcheck --all-databases --check-upgrade

-> check the target version is valid for upgrade

    mysqlsh -- util check-for-server-upgrade root@localhost --target-version=8.0.30 
            --output-format=JSON --config-path=/etc/mysql/my.cnf

-> downlod MYSQL 8.0.30 version binary 

   tar -xvf mysql-8.0.30-linux-glibc2.12-x86_64.tar.xz

->  shutdown mysqld service , unlink old binary and link to new binary

     /etc/init.d/mysql.server stop
     cd /usr/local
     unlink mysql
     ln -s /home/mysql/mysql-8.0.30-linux-glibc2.12-x86_64 mysql

-> start mysql , check and verify

   /etc/init.d/mysql.server start
   pidof mysqld
   netstat -ntlp | grep 3306

In [ ]:
Major version upgrade from 5.7 to 8.0.10 ( rpm packages )
=================================================================

-> check issues in the current database objects if any 

     mysqlcheck --all-databases --check-upgrade

-> check the target version is valid for upgrade

    mysqlsh -- util check-for-server-upgrade root@localhost --target-version=8.0.10 
            --output-format=JSON --config-path=/etc/mysql/my.cnf

-> downlod MYSQL 8.0.10 rpm packages 

->  shutdown mysqld service , and remove 5.7 packages

    systemctl stop mysqld
    pidof mysqld

    yum remove mysql-community-server-5.7.20-1.el7.x86_64
    yum remove mysql-community-common-5.7.20-1.el7.x86_64

-> install new 8.0.10 packages

   yum localinstall mysql-community-*

   rpm -qa | grep mysql

-> start mysql service and run upgrade ( prior to 8.0.16 )

   systemctl start mysqld

   mysql_upgrade

   select @@hostname , @@version;


In [ ]:
Downgrading MySQL 
=======================================

-> Downgrade from MySQL 8.0 to MySQL 5.7 is not supported.

-> Downgrade from a MySQL 8.0 release to a previous MySQL 8.0 release is also not supported.

-> Only supported alternative is to restore from a backup taken before upgrade.


In [ ]:
MySQL PROGRAMS 
================================

mysqld - is the main server program that does most of work in mysql installation.

mysqld_safe - is the server startup script and starts mysqld.

mysql.server - is also a server startup script . this invokes mysqld_safe to start the mysql server.

mysqld_multi - is a program that helps to start multiple mysql servers.

mysql_secure_installation - is a program that helps in securing mysql server.

mysql_ssl_rsa_setup - creates ssl certificates and key files for secure connections.

mysql_tzinfo_to_sql - this program loads the time zone tables in the mysql database from the 
                      host zoneinfo database.

mysql_upgrade - prior to 8.0.16 this program is used during upgrade after the new version is installed.

mysql - command line tool to access the mysql server.

mysqladmin - program used for administrative tasks.

mysqlcheck - program is used to repair , analyse and optimise tables.

mysqldump - program is used to dump mysql database into a file such as sql , text or xml.

mysqlimport - program is used to import text files into tables.

mysqlpump - similar to mysqldump with extra features.

mysqlsh - is a command line tool with extra scripting abilites like javascript and python.

mysqlshow - displays information about databases , tables ,columns and indexes.

mysqlslap - load emulation client. 

mysql_config_editor - used to store credentials in secure and encrypted form.

mysqlbinlog - used to read statements from binlog. this can be used during crash recovery and PITR.

mysqldumpslow - used to summarize the contents of slow query log.


In [ ]:

MySQL Config Editor 
=============================

-- helps in storing encrypted database credentials.
-- with filname .mylogin.cnf in users home directory.
-- default group is [client] and for any other group --login-path has to be specicified.

mysql_config_editor set -u root -p

mysql_config_editor print

mysql_config_editor print --all

mysql_config_editor set --login-path=dbadmin_client -u db_admin -p

mysql_config_editor reset


mysql_tzinfo_to_sql program
=======================================

-- loads time zone data from zoneinfo database into mysql database.
-- location in linux is /usr/share/zoneinfo

mysql_tzinfo_to_sql /usr/share/zoneinfo/ | mysql mysql


mysqlshow program
================================

-- displays database , table and column information.

mysqlshow employeesdb

mysqlshow employeesdb employees

mysqlshow employeesdb employees emp_id


mysqlcheck program
=======================================

-- is a table maintenence program , it checks , repairs , oprtimize or analyse the table.
-- table will be locked during the opertion.
-- -c check , -a analyze , -o optimize , -r repair.
-- -c is default.

mysqlcheck employeesdb employees

mysqlcheck -o -a employeesdb employees

mysqlcheck -r employeesdb employees


mysqlimport program
===================================

-- is a data import program , takes .txt file as input with tab delimited and table name as filename.
-- can be loaded only with location of secure_file_priv variable.

mysqlimport employeesdb staff_data.txt


MySQL Executing SQL Files 
==========================================

mysql> source create_table_staff.sql

mysql> \. create_table_staff.sql

mysql < create_table_staff.sql 

[mysqluser@mysqlserver ~]$ cat create_table_staff.sh
mysql < $1

[mysqluser@mysqlserver ~]$ ./create_table_staff.sh create_table_staff.sql &


Mysqladmin Program
===================================

-- is a program to carry out admin tasks like shutdown , create/drop database , check status , ping , 
   start/stop replica.

mysqladmin ping

mysqladmin status

mysqladmin shutdown

mysqladmin create employeesdb


mysqlpump program 
==============================

-- this is similar to mysqldump which takes logical backup
-- extra features include parallelism which taking backup 
-- dump of just user accounts 

mysqlpump --parallel-schemas=ordersdb,productsdb --parallel-schemas=sakila > db_dump.sql

mysqlpump --exclude-databases=% --users > dump_users.sql


mysqlslap program 
============================

-- is a program to emulate client load on the mysql server and report timing of each stage.
-- it works as if mutiple clients are accessing the server.

mysqlslap --concurrency=5 --iterations=20 --number-int-cols=2 --number-char-cols=3 --auto-generate-sql
                     

mysqlbinlog program 
=============================

-- binary logs or binlogs records changes in the database in binary format.
-- binary logs are used in crash recovery , point-in-time recovery and replication.
-- mysqlbinlog can be used to display the contents of this binary log or the relay log in replica.


output binlog content on screen
--------------------------------------

mysqlbinlog binlog.000006

output binlog content to a file 
--------------------------------------

mysqlbinlog binlog.000006 > /tmp/binlog6.sql

replace database name in the binlog to a different database name
-------------------------------------------------------------------------------

mysqlbinlog --rewrite-db='airportdb->airports' binlog.000006 > /tmp/binlog6_airports.sql

skip gtids inside the binlog 
-----------------------------------------

mysqlbinlog --skip-gtids binlog.000006 > /tmp/binlog6_skipgtid.sql

read binlog from a specicified date time 
---------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --start-datetime="2024-05-28 03:56:00"  binlog.000006 > 
                         /tmp/binlog6_startdatetime.sql

read binlog from a specified log position 
------------------------------------------------

mysqlbinlog --start-position=14006  binlog.000006 > /tmp/binlog6_startpos.sql

read binlog until a specified date time 
------------------------------------------------

mysqlbinlog --stop-datetime="2024-05-28 03:56:00"  binlog.000006 > /tmp/binlog6_stopdatetime.sql

read binlog until a specified log position 
----------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --stop-position=14006  binlog.000006 > /tmp/binlog6_stoppos.sql

read binlog from a specified datetime until a specicified date time
---------------------------------------------------------------------------

 mysqlbinlog --start_datetime="2024-05-28 03:53:00" --stop-datetime="2024-05-28 03:57:00"  
             binlog.000006 > /tmp/binlog6_startstopdatetime.sql

read binlog from a specified log position until a specicified log position
---------------------------------------------------------------------------

mysqlbinlog --start-position=4348 --stop-position=14006  binlog.000006 > /tmp/binlog6_startstoppos.sql

read binlog from a specified log position until a specicified datetime
---------------------------------------------------------------------------

mysqlbinlog --start-position=4348 --stop-datetime="2024-05-28 03:57:00"  binlog.000006 > 
            /tmp/binlog6_startposstopdatetime.sql

read mutile binlog files 
----------------------------------------------

mysqlbinlog binlog.000006 binlog.000007 binlog.000008 > /tmp/binlog6_7_8.sql
 
mysqlbinlog binlog.00000[6-8] > /tmp/binlog_6_7_8.sql


by default row events are displayed as base64 encoded , to show it as sql use --verbose or -v 
------------------------------------------------------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog -v binlog.000006 > /tmp/binlog6_verbose.sql


mysqlbinlog can be used to backup the binlog to different location ( raw binary format )
--------------------------------------------------------------------------------------------

mysqlbinlog --raw --read-from-remote-server binlog.000003 binlog.000004 binlog.000005

mysqlbinlog --raw --read-from-remote-server --result-file=/home/mysqluser/backups/bosp binlog.000003 binlog.000004

mysqlbinlog --raw --read-from-remote-server --result-file=/home/mysqluser/backups/bosp --to-last-log binlog.000003

mysqlbinlog --raw --read-from-remote-server --result-file=/home/mysqluser/backups/bosp --stop-never binlog.000007 &


mysqldumpslow 
===================================

-- is used to summarize slow query logs.

mysqldumpslow /var/lib/mysql/bospprod01-slow.log


In [ ]:
mysqldump utility 
==================================

-- is a command line utility to take logical backups.
-- this will dump a mysql database / table into SQL , TEXT or XML format.

syntax : 

mysqldump [options] db_name [tbl_name , ]

mysqldump [options] --databases db_name1 db_name2

mysqldump [options] --all-databases


single database backup 
---------------------------

mysqldump -u root -p ClassicModels > /home/mysqluser/backups/ClassicModels.sql


only definition backup without data 
----------------------------------------

mysqldump -u root -p --no-data sakila > /home/mysqluser/backups/sakila_db_backup_definition.sql 


only data backup without definitions 
------------------------------------------

mysqldump -u root -p --no-create-info sakila > /home/mysqluser/backups/sakila_db_backup_dataonly.sql


specifying defaults for mysqldump 
--------------------------------------------

[root@MySqlServer1 backups]# cat /etc/my.cnf
[mysqldump]
host=localhost
user=db_admin
password=Dbadmin@2024

[root@MySqlServer1 backups]# mysqldump --print-defaults

mysqldump would have been started with the following arguments:
--host=localhost --user=db_admin --password=***** 
[root@MySqlServer1 backups]# 


all database backup
---------------------------

mysqldump --all-databases > dump_alldbs_30apr2024.sql


mutiple databases backup 
-------------------------------

mysqldump -u root -p --databases sample sakila > dbdump_sample_sakila.sql


single table backup 
----------------------------

mysqldump -u root -p sample sample_employees > sample_sample_employees_bkp.sql


mutiple tables backup 
-------------------------------

mysqldump -u root -p sample sample_employees sample_customers sample_sales > 
sample_sample_tables_bkp.sql


compress the backup 
---------------------------------

mysqldump -u root -p --databases sample sakila | gzip > dump_db_sample_sakila.sql.gz


Create a consistent data dump without locking tables
----------------------------------------------------------

mysqldump --single-transaction --skip-lock-tables sakila > dump_consistent_sakila_db_30apr2024.sql


Dump a remote database
--------------------------------------

mysqldump -h localhost --port 3306 -u db_admin -p sakila > dump_remote_sakila_db_30apr2024.sql


Use where clause with mysqldump
---------------------------------------

mysqldump sample sample_employees --where="employee_id > 1000" > dump_sample_employees_custid1000.sql


Limit number of records in a dump ( There is no flag for limitting but we can use where clause)
--------------------------------------------------------------------------------------------------

mysqldump sample --where='1 limit 500' > dump_sample_500records.sql


Dump database with excluding table / tables
------------------------------------------------

mysqldump sample --ignore-table sample.sample_sales > dump_sample_exclude_sales.sql

mysqldump sample --ignore-table sample.sample_sales --ignore-table sample.sample_customers >
 dump_sample_exclude_sales_customers.sql


Dump database with routines ( by default mysqldump does not include routines)
---------------------------------------------------------------------------------

mysqldump Northwind --routines > dump_northwind.sql


Dump large tables with quick option 
----------------------------------------------------------------------------------

mysqldump --comments --quick airportdb > airportdb_dump_quick.sql



Restoring a dump file 
============================================

mysql -u root -p sample < sample_sample_tables_bkp.sql


copy data from one host to another using mysqldump 
============================================================

mysqldump ordersdb | mysql -h 192.168.153.137 -u db_admin -p******** orders


In [ ]:
Server configuration
======================================

mysqld has many command options and system variables that can be set up at startup.

To determine the default command option and system variable values we can execute the below :

mysqld --verbose --help 

mysqld --verbose --help | grep innodb-buffer-pool-size

mysql> show variables;

mysql> show variables like '%innodb_buffer%';


mysqld --validate-config can be used to check for any problems or issues in the option file.

In [ ]:
SQL MODES 
===========================

Modes afftect the SQL syntax and the data validation checks performed by Mysql server.

Default sql_mode is ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION

sql_mode can be set at runtime or in the option file ( my.cnf )

SET GLOBAL sql_mode = "modes"

SET SESSION sql_mode = "modes"

In [ ]:
Connection Management 
==============================================


connection manager threads handle client connection requests.
connection manager consults thread cache before creating a new thread.
when a connection ends , its thread is returned to thread cache.

thread_cache_size determines the size of thread cache , if its 0 then its disabled.

mysql> show variables like '%thread_cache%';

mysql> show global status like 'threads%';



The max number of connections allowed is controlled by max_connections variable.

mysql> show variables like '%max_connections%';


The maximum number of client connections depends on several factors :

-- the quality of thread library , amount of RAM available.
-- workload from each connection.
-- number of file descriptors available.


check default number of file descriptors available in linux for users 
-----------------------------------------------------------------------

ulimit -n  ( default value 1024 )

increase the number of file descriptors to match max_connections
---------------------------------------------------------------------

cat /etc/security/limits.conf | grep -v '^#'

*                soft    nofile          5000
*                hard    nofile          5000

increase the max_connections in mysql my.cnf 
----------------------------------------------------

[root@mysqlprod02 ~]# cat /etc/my.cnf | grep max-connection
max-connections = 1500
[root@mysqlprod02 ~]# 


DNS LOOKUP and HOST CACHE 
=========================================

The mysql server maintains a in-memory host cache that contains info about clients IP , hostname and error information.

host_cache_size | 628
max_connect_errors | 100


Mysql server time zone support 
==========================================

-- when the server starts , it attempts to determine the time zone of the host machine and uses it to set the 
   system_time_zone system varaible.

-- you can set the default-time-zone in option file OR set global and session time_zone variable at run-time 

-- you can load the time zone tables in mysql database with zoneinfo database of the OS using mysql_tzinfo_to_sql program

-- load from single time zone file 

    mysql_tzinfo_to_sql /usr/share/zoneinfo/UTC UTC | mysql mysql

-- load from zoneinfo directory 

    mysql_tzinfo_to_sql /usr/share/zoneinfo/ | mysql mysql

mysql> set time_zone = 'UTC';

mysql> set global time_zone = 'UTC';

In [ ]:
General query log and slow query log output destinations
=====================================================================


-- log output destination can be set in option file or at run-time using log-output variable ( TABLE , FILE , NONE )

mysql> select @@GLOBAL.log_output ,  @@GLOBAL.general_log , @@GLOBAL.slow_query_log;


--set log-output to TABLE and enable general log and slow query log 

mysql> set global log_output = 'TABLE';

mysql> set global general_log = 'ON';

mysql> set global slow_query_log = 'ON';


mysql> select * from mysql.general_log \G

mysql> select * from mysql.slow_log \G


--set log-output to both TABLE and FILE

mysql> set global log_output = 'TABLE,FILE';


General Query Log
===========================

  - logs each statement received from clients , binary log does not contain select statements.
  - by default its is disabled , enable by setting general_log = 1 and general_log_file = file_name.
  - it can grow exponentially as it logs every statement.


  mysql> show variables like '%general_log%';


Slow Query Log
=========================================

  - logs queries that takes more than long_query_time seconds to execute and require atleast 
    min_examined_row_limit rows to be examined.
  - is disabled by default , enable by setting slow_query_log = 1 and slow_query_log_file = file_name.
  - mysqldumpslow can be used to summarise the contents.
  - by default admin statements and queries that do not use index are not captured.

  [root@MySqlServer1 ~]# cat /etc/my.cnf | grep query
        slow_query_log = 1
        slow_query_log_file = /var/lib/mysql/MySqlServer1-slow.log
        long_query_time = 5


Error Log
=================================================

    - logs server startup and shutdown time.
    - logs errors , warnings that occur during startup or shutdown of mysqld.
    - logs table check and repair logs.

mysql> show variables like 'log_error';

In [ ]:
Binary log  -- is a set of log files that contain information about data modifications made to mysql database.
            -- is used for replication . the master server sends the events contained in its binary log to
               its slaves , which executes those events to make the same data changes that wer made in master.
            -- is used during database recovery opeartions( point in time recovery)

Types of Binary log:
    
    statement-based logging - conatins sql statements.
    row-based logging - contains changes done to individual rows. ( default )
    mixed logging - by default statement-based but switches to row based automatically as necessary.

mysqlbinlog utility can be used to print binary or relay log in readable format.

log_bin                   | ON                                      
log_bin_basename          | /var/lib/mysql/mysqlprod02_binlog     ( binlog )  
log_bin_index             | /var/lib/mysql/mysqlprod02_binlog.index (binlog.index)
binlog_format             | MIXED  ( STATEMENT OR ROW )


mysql> show binary logs;

mysql> show binlog events;

mysql> show binlog events in 'mysqlprod02_binlog.000011';


you can also enable compression for binary logs which will save storage space.


binlog_transaction_compression            | OFF   |
binlog_transaction_compression_level_zstd | 3     |

In [ ]:
MySQL Physical Cold Backup
=============================================

-- Cold backup involves physically copying database related files to backup device
   taking the database down.
-- for this the mysql database server should be clean shutdown.

files required in cold backup
==========================================

-- all the files in datadir.
-- all the files of system tablespaces.
-- all the configuration/option files.

files not required in cold backup
============================================

-- redo log files , undo log files , bin log files 
-- temp files , double write buffer files.


clean shutdown the database
-----------------------------------------

mysql> set global innodb_fast_shutdown = 0;

[root@mysqlserver mysql]# systemctl stop mysqld


copy files to backup location 
------------------------------------------

[root@mysqlserver mysql]# cp -r /var/lib/mysql/data /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -r /var/lib/mysql/systemdata /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -p /etc/mysql/my.cnf /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -p /var/lib/mysql-files/*.cnf /tmp/mysql_cold_backup/


to restore mysql instance from the cold backup , remove all the files 
-------------------------------------------------------------------------

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/binlog/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/systemdata/*


copy the files from backup
-------------------------------------

[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/data/* /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/systemdata/* /var/lib/mysql/systemdata/

[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/systemdata/


start the mysql instance 
-------------------------------------

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 

In [ ]:
Logical Backups
==============================

-- does not involve copying of physical files.
-- all databases , some databases and table level backup can be taken.
-- utility have to be used like mysqldump amd mysqlpump.
-- takes backup in .sql format , text delimited or XML format.

[mysqluser@mysqlserver backups]$ mysqldump --verbose --help

Dumping structure and contents of MySQL databases and tables.
Usage: mysqldump [OPTIONS] database [tables]
OR     mysqldump [OPTIONS] --databases [OPTIONS] DB1 [DB2 DB3...]
OR     mysqldump [OPTIONS] --all-databases [OPTIONS]


Single Table Backup
-----------------------

mysqldump sakila actor > sakilaDB_actorTable_bkp.sql


Whole Single Database Backup 
-------------------------------

mysqldump sakila > sakilaDB_bkp.sql


Two databases backup
--------------------------

mysqldump --databases posdb sakila > posdb_sakila_bkp.sql


All databases backup 
-----------------------------

mysqldump --all-databases > all_dbs_bkp.sql


Database backup with skipping some tables
-------------------------------------------------

mysqldump sakila --ignore-table=sakila.actor --ignore-table=sakila.film_actor > 
          sakiladb_skipactor_bkp.sql


Multiple database backup with skipping some tables
-------------------------------------------------------

mysqldump --databases sakila posdb --ignore-table=sakila.actor --ignore-table=sakila.film_actor 
          --ignore-table=posdb.emp_salaries > sakiladb_posdb_ignoretables_bkp.sql


consistent backup 
-------------------------------------------------

mysqldump --single-transaction > all_dbs_consistent_bkp.sql


consistent backup without locking tables
-------------------------------------------------

mysqldump --all-databases --single-transaction --skip-lock-tables > all_dbs_consistent_bkp_dump.sql


definition backup without data 
--------------------------------------------

mysqldump -u root -p --no-data sakila > /home/mysqluser/backups/sakila_db_backup_definition.sql


only data backup without definition ( create statements )
------------------------------------------------------------------

mysqldump -u root -p --no-create-info sakila > /home/mysqluser/backups/sakila_db_backup_dataonly.sql


backup databases with compressed backup file 
------------------------------------------------------

mysqldump -u root -p --databases sample sakila | gzip > dump_db_sample_sakila.sql.gz


Dump large tables with quick option 
-----------------------------------------------

mysqldump --comments --quick airportdb > airportdb_dump_quick.sql


copy data from one host to another using mysqldump
-------------------------------------------------------

mysqldump ordersdb | mysql -h 192.168.153.137 -u db_admin -p******** orders


Dump a remote database
--------------------------------------------

mysqldump -h bosprod01 --port 3306 -u db_admin -p sakila > dump_bosprod01_sakiladb.sql


Use where clause to filter data
-----------------------------------------

mysqldump sample sample_employees --where="employee_id > 1000" > dump_sample_employees_custid1000.sql


Limit backup file using where clause
------------------------------------------------

mysqldump sample --where='1 limit 500' > dump_sample_500records.sql


Dump database with routines ( by default mysqldump does not include routines)
---------------------------------------------------------------------------------

mysqldump Northwind --routines > dump_northwind.sql




Single Table Restore from backup taken for single table
---------------------------------------------------------------

mysql> drop table actor;

mysql -u db_admin -p sakila < sakilaDB_actorTable_bkp.sql


Single Table restore from backup taken for full database
--------------------------------------------------------------

ALL CREATE TABLE statements start with DROP TABLE IF EXISTS and then CREATE TABLE and then
INSERT STATEMENTS and it will end with keyword UNLOCK TABLES

sed -n -e '/DROP TABLE.*`film_actor`/,/UNLOCK TABLES/p' sakilaDB_bkp.sql > sakilaDB_filmactor_bkp.sql

drop table actor ;

mysql sakila < sakilaDB_filmactor_bkp.sql


Database restore from backup taken for a single database
---------------------------------------------------------------------

mysql> drop database sakila;

mysql sakila < sakilaDB_bkp.sql


Restore a single database from all database backup
-------------------------------------------------------

cat dump_alldatabases.sql | grep 'CREATE DATABASE' | grep -v mysql

mysql> drop database ordersdb;

mysql> drop database sakila;

If the create database is in between the dump 
----------------------------------------------------------

sed -n -e '/CREATE DATABASE.*`ordersdb`/,/CREATE DATABASE/p' dump_alldatabases.sql > ordersdb_bkp.sql

remove the last create datbase statement in ordersdb_bkp.sql

mysql < ordersdb_bkp.sql

If the create database is the last 
----------------------------------------------------------

sed -n -e '/CREATE DATABASE.*`sakila`/,/Dump completed/p' dump_alldatabases.sql > sakiladb_bkp.sql

mysql < sakiladb_bkp.sql
                            